In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')
from src.encode_data import *
from src.midi_data import *
# from src.data_sources import process_parallel, transform_csv_row
from src.midi_transform import *
from concurrent.futures import ProcessPoolExecutor
from fastprogress.fastprogress import master_bar, progress_bar

In [3]:
from tqdm import tqdm

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
version = 'v11'
data_path = Path('data/midi')
version_path = data_path/version

In [5]:
import pandas as pd

In [6]:
out_dir = 'midi_encode'
source_csv = version_path/out_dir/f'{out_dir}.csv'
source_csv.parent.mkdir(parents=True, exist_ok=True)
source_csv

PosixPath('data/midi/v11/midi_encode/midi_encode.csv')

In [7]:
num_comps = 2 # note, duration
cutoff = 4 # max instruments

In [8]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ht_bpm,md5,numpy,ht_time_signature,midi_title,source,artist,ht_offset,section,song_url,mxl,ht_key,parts,title,genres,midi,ht_mode
0,128.0,bf1f29e5ff84e3e93e37fb873bfb590e,midi_encode/hooktheory/pianoroll/w/wayne-sharp...,4.0,yu-gi-oh3,hooktheory,wayne-sharpe,0.0,chorus,https://www.hooktheory.com/theorytab/view/wayn...,NaN,C,"intro,chorus",yu-gi-oh-theme-song,NaN,midi_sources/hooktheory/pianoroll/w/wayne-shar...,1.0
1,85.0,055f80ad67f64edb14a85ca8fbfe8c29,midi_encode/hooktheory/pianoroll/w/wayne-sharp...,3.0,yu-gi-oh,hooktheory,wayne-sharpe,0.0,intro,https://www.hooktheory.com/theorytab/view/wayn...,NaN,C,"intro,chorus",yu-gi-oh-theme-song,NaN,midi_sources/hooktheory/pianoroll/w/wayne-shar...,1.0
2,96.0,197f96f5d181f6ce1e2c5ab04ac1ff87,midi_encode/hooktheory/pianoroll/w/what-a-day/...,4.0,kiefer,hooktheory,what-a-day,-5.0,chorus,https://www.hooktheory.com/theorytab/view/what...,NaN,D,chorus,kiefer,Jazz,midi_sources/hooktheory/pianoroll/w/what-a-day...,6.0
3,152.0,9e7ce13a35f1314423a9a6d5a5287a4a,midi_encode/hooktheory/pianoroll/w/whiteflame/...,4.0,senbonzakura - pre-Pre-Chorus,hooktheory,whiteflame,-5.0,pre-chorus,https://www.hooktheory.com/theorytab/view/whit...,NaN,D,"verse,pre-chorus,chorus",senbonzakura,"J-Pop,Pop",midi_sources/hooktheory/pianoroll/w/whiteflame...,6.0
4,152.0,d5aaf79d0989222f1362f9f46c540a27,midi_encode/hooktheory/pianoroll/w/whiteflame/...,4.0,Senbonzakura,hooktheory,whiteflame,-5.0,verse,https://www.hooktheory.com/theorytab/view/whit...,NaN,D,"verse,pre-chorus,chorus",senbonzakura,"J-Pop,Pop",midi_sources/hooktheory/pianoroll/w/whiteflame...,6.0


In [9]:
ecomp_records = df.loc[df.source == 'ecomp'].to_dict(orient='records')

In [10]:
all_records = df.to_dict(orient='records')

### Via Data Sources

In [11]:
import concurrent

In [12]:

def process_all(func, arr, total=None, max_workers=None, timeout=None):
    "Process array in parallel"
    if total is None: total = len(arr)
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(func,o) for i,o in enumerate(arr)]
        for f in tqdm(concurrent.futures.as_completed(futures, timeout=timeout), total=total):
            res = f.result(timeout=timeout)

### Need better midi formatting

In [72]:
# metadata = all_records[0]

# # input_path = version_path/'old'/metadata['numpy']
# input_path = version_path/metadata['numpy']
# output_path = version_path/metadata['numpy']
# # if output_path.exists(): return
# output_path.parent.mkdir(parents=True, exist_ok=True)
# np_array = np.load(input_path)


# np_array[np_array[:,0] == (VALTSEP+ENC_OFFSET), 1] += 1
# # np.save(output_path)

# np_array

In [27]:
# WARNING: classic_piano source is called classical_piano, lmd_clean is lmd, 130kreddit is reddit
sources = ['hooktheory', 'hooktheory_c', 'freemidi', 'midiworld', 'ecomp', 'cprato', 'classical_piano', 'musescore', 'wikifonia', 'lmd', 'reddit']

In [22]:
def transform_func(metadata):
    input_path = version_path/'old'/metadata['numpy']
    output_path = version_path/metadata['numpy']
    if not input_path.exists(): return
    if output_path.exists(): return
    output_path.parent.mkdir(parents=True, exist_ok=True)
    np_array = np.load(input_path)
    np_array[0, 1] = ENC_OFFSET + 128 + sources.index(metadata['source']) + 1 # set category metadata
    np_array[np_array[:,0] == (VALTSEP+ENC_OFFSET), 1] += 1
    np.save(output_path, np_array)
#     print(output_path)

In [16]:
# # sanity check
# import random
# for r in random.sample(ecomp_records, 500):
#     transform_func(r)
        

In [17]:
### AS TOD: Fix 'info channel is not channel 0' error

In [18]:
# all_records = all_records[60000:]

In [28]:
processed = process_all(transform_func, all_records)





  0%|          | 0/174994 [00:00<?, ?it/s]



  0%|          | 1/174994 [00:00<28:10:27,  1.73it/s]



  0%|          | 271/174994 [00:00<19:41:48,  2.46it/s]



  0%|          | 609/174994 [00:00<13:45:55,  3.52it/s]



  1%|          | 948/174994 [00:00<9:37:16,  5.02it/s] 



  1%|          | 1285/174994 [00:00<6:43:34,  7.17it/s]



  1%|          | 1618/174994 [00:01<4:42:13, 10.24it/s]



  1%|          | 1963/174994 [00:01<3:17:24, 14.61it/s]



  1%|▏         | 2308/174994 [00:01<2:18:09, 20.83it/s]



  2%|▏         | 2653/174994 [00:01<1:36:46, 29.68it/s]



  2%|▏         | 2997/174994 [00:01<1:07:51, 42.25it/s]



  2%|▏         | 3336/174994 [00:01<47:39, 60.03it/s]  



  2%|▏         | 3674/174994 [00:01<33:32, 85.11it/s]



  2%|▏         | 4024/174994 [00:01<23:40, 120.33it/s]



  2%|▏         | 4370/174994 [00:01<16:47, 169.35it/s]



  3%|▎         | 4717/174994 [00:01<11:58, 236.97it/s]



  3%|▎         | 5067/174994 [00:02<08:36, 328.98it/s]



  3%|▎        

 22%|██▏       | 38781/174994 [00:11<00:39, 3439.13it/s]



 22%|██▏       | 39130/174994 [00:12<00:39, 3452.76it/s]



 23%|██▎       | 39476/174994 [00:12<00:39, 3453.47it/s]



 23%|██▎       | 39822/174994 [00:12<00:39, 3452.15it/s]



 23%|██▎       | 40168/174994 [00:12<00:39, 3441.94it/s]



 23%|██▎       | 40513/174994 [00:12<00:39, 3440.06it/s]



 23%|██▎       | 40862/174994 [00:12<00:38, 3454.32it/s]



 24%|██▎       | 41209/174994 [00:12<00:38, 3456.78it/s]



 24%|██▎       | 41555/174994 [00:12<00:38, 3457.26it/s]



 24%|██▍       | 41901/174994 [00:12<00:38, 3445.66it/s]



 24%|██▍       | 42246/174994 [00:12<00:38, 3444.29it/s]



 24%|██▍       | 42591/174994 [00:13<00:38, 3429.45it/s]



 25%|██▍       | 42937/174994 [00:13<00:38, 3438.01it/s]



 25%|██▍       | 43282/174994 [00:13<00:38, 3432.68it/s]



 25%|██▍       | 43628/174994 [00:13<00:38, 3440.56it/s]



 25%|██▌       | 43973/174994 [00:13<00:38, 3442.21it/s]



 25%|██▌       | 44318/174994 [00:13<00:

 45%|████▍     | 78013/174994 [00:23<00:27, 3497.50it/s]



 45%|████▍     | 78363/174994 [00:23<00:27, 3495.91it/s]



 45%|████▍     | 78713/174994 [00:23<00:27, 3490.43it/s]



 45%|████▌     | 79063/174994 [00:23<00:27, 3472.02it/s]



 45%|████▌     | 79411/174994 [00:23<00:27, 3471.83it/s]



 46%|████▌     | 79759/174994 [00:23<00:27, 3468.63it/s]



 46%|████▌     | 80112/174994 [00:23<00:27, 3486.25it/s]



 46%|████▌     | 80461/174994 [00:24<00:28, 3368.63it/s]



 46%|████▌     | 80799/174994 [00:24<00:28, 3307.17it/s]



 46%|████▋     | 81131/174994 [00:24<00:28, 3308.61it/s]



 47%|████▋     | 81465/174994 [00:24<00:28, 3317.44it/s]



 47%|████▋     | 81803/174994 [00:24<00:27, 3333.94it/s]



 47%|████▋     | 82141/174994 [00:24<00:27, 3347.31it/s]



 47%|████▋     | 82478/174994 [00:24<00:27, 3351.66it/s]



 47%|████▋     | 82822/174994 [00:24<00:27, 3375.21it/s]



 48%|████▊     | 83163/174994 [00:24<00:27, 3379.49it/s]



 48%|████▊     | 83502/174994 [00:24<00:

 67%|██████▋   | 116637/174994 [00:34<00:17, 3421.70it/s]



 67%|██████▋   | 116980/174994 [00:34<00:17, 3409.49it/s]



 67%|██████▋   | 117321/174994 [00:34<00:17, 3381.43it/s]



 67%|██████▋   | 117662/174994 [00:35<00:16, 3388.17it/s]



 67%|██████▋   | 118001/174994 [00:35<00:16, 3383.31it/s]



 68%|██████▊   | 118340/174994 [00:35<00:16, 3353.72it/s]



 68%|██████▊   | 118676/174994 [00:35<00:16, 3339.67it/s]



 68%|██████▊   | 119015/174994 [00:35<00:16, 3349.09it/s]



 68%|██████▊   | 119356/174994 [00:35<00:16, 3365.21it/s]



 68%|██████▊   | 119697/174994 [00:35<00:16, 3376.42it/s]



 69%|██████▊   | 120037/174994 [00:35<00:16, 3382.41it/s]



 69%|██████▉   | 120376/174994 [00:35<00:16, 3383.91it/s]



 69%|██████▉   | 120717/174994 [00:35<00:16, 3390.53it/s]



 69%|██████▉   | 121061/174994 [00:36<00:15, 3403.88it/s]



 69%|██████▉   | 121402/174994 [00:36<00:15, 3396.92it/s]



 70%|██████▉   | 121742/174994 [00:36<00:15, 3391.48it/s]



 70%|██████▉   | 122082/

 89%|████████▊ | 154945/174994 [00:45<00:06, 3339.01it/s]



 89%|████████▊ | 155281/174994 [00:46<00:05, 3343.70it/s]



 89%|████████▉ | 155616/174994 [00:46<00:05, 3297.84it/s]



 89%|████████▉ | 155955/174994 [00:46<00:05, 3323.84it/s]



 89%|████████▉ | 156295/174994 [00:46<00:05, 3345.31it/s]



 90%|████████▉ | 156630/174994 [00:46<00:05, 3339.74it/s]



 90%|████████▉ | 156977/174994 [00:46<00:05, 3366.87it/s]



 90%|████████▉ | 157320/174994 [00:46<00:05, 3383.33it/s]



 90%|█████████ | 157662/174994 [00:46<00:05, 3393.47it/s]



 90%|█████████ | 158005/174994 [00:46<00:04, 3403.21it/s]



 90%|█████████ | 158346/174994 [00:47<00:04, 3397.49it/s]



 91%|█████████ | 158690/174994 [00:47<00:04, 3405.99it/s]



 91%|█████████ | 159031/174994 [00:47<00:04, 3400.96it/s]



 91%|█████████ | 159372/174994 [00:47<00:04, 3398.55it/s]



 91%|█████████▏| 159713/174994 [00:47<00:04, 3400.16it/s]



 91%|█████████▏| 160054/174994 [00:47<00:04, 3340.54it/s]



 92%|█████████▏| 160389/